# **Content Based Collaborative Filtering**

In [1]:
import pandas as pd
import numpy as np

In [2]:
dataframe_robust_ratings = pd.read_csv('Preprocessed/robust_scaled_ratings.csv')
LDA_dataframe_jokes = pd.read_csv('Preprocessed/LDA_dataframe_jokes.csv')

## **Functions**

In [3]:
def get_intrested_jokes(user_id,df,jokes_df,no_jokes,interest_threshold,joke_matpic):
    """
    Finds the top "no_jokes" intrested jokes for active user

    Args:
        user_id (int): id of active user
        df (pd.Dataframe): Dataframe of joke_ratings for all users(normalized)
        jokes_df (pd.DataFrame): Dataframe of jokes after cleaning
        no_jokes (int): no of jokes
        interest_threshold (float b/w -1 to 1): rating threshold of intrest
        joke_matpic (pd.DataFrame): Dataframe of joke main topics

    Returns:
        list<int>: list of joke ids of the top "no_jokes" intrested jokes for active user
    """
    user_rating=df.loc[user_id-1]
    # generation of intrested topics of jokes for the user
    interested_topics=set([joke_matpic.loc[column-2] for column in df.columns[2:102] if(user_rating[column]<interest_threshold)])
    
    # finding new jokes for recommending
    new_jokes=[] # intrested and unrated
    old_jokes=[] # intrested and rated
    other_jokes=[] # unintrested and unrated
    worst_jokes=[] # unintrested and rated
    for i in range(100):
        if jokes_df['main_topic'][i] in interested_topics:
            if user_rating[i+2] !=user_rating[i+2]:
                new_jokes.append(i)
            else:
                old_jokes.append(i)
        elif user_rating[i+2] !=user_rating[i+2]:
            other_jokes.append(i)
        else:
            worst_jokes.append(i)
            
    #add new_jokes if needed
    if(len(new_jokes)<no_jokes):
        new_jokes+=old_jokes
        if(len(new_jokes)<no_jokes):
            if(no_jokes-len(new_jokes)>len(other_jokes)):
                new_jokes+=other_jokes
                new_jokes+=(sorted(worst_jokes,key=(lambda x: user_rating[x+2]))[::-1])[:no_jokes-len(new_jokes)]
            else:
                for i in np.random.choice(len(other_jokes),no_jokes-len(new_jokes),replace=False):
                    new_jokes.append(other_jokes[i])
                
    
    # choose randomly "no_jokes" no of jokes
    for i in np.random.choice(len(new_jokes), no_jokes, replace=False):
        new_jokes.append(new_jokes[i])
    return new_jokes[(-1*no_jokes):]
            
def recommend_cbf(user_id,df,jokes_df):
    """
    Recommends jokes based on cbf approach for given user
    
    Args:
        user_id (int): id of active user
        df (pd.Dataframe): Dataframe of joke_ratings for all users(normalized)
        jokes_df (pd.DataFrame): Dataframe of jokes after cleaning
    Returns:
        None
    """
    no_jokes=20 # no of jokes to recommend
    interest_threshold=0.8 # threshold for chossing rating of interest
    joke_matpic=jokes_df['main_topic'] # main topics of joke for finding interested topics of user 
    new_jokes=get_intrested_jokes(user_id,df,jokes_df,no_jokes,interest_threshold,joke_matpic)
    
    print("Recommending Jokes for user {}".format(user_id))
    for i,k in zip(range(no_jokes),new_jokes):
        print("Joke {}: {}".format(i+1,jokes_df['joke'][k]))

## **Results**

In [11]:
user_id = 68388
recommend_cbf(user_id,dataframe_robust_ratings,LDA_dataframe_jokes)

Recommending Jokes for user 68388
Joke 1:  How many Irishmen does it take to change a lightbulb?   Two, one to hold the lightbulb and the other to drink until the room spins.  
Joke 2:  Just a thought ..   Before criticizing someone, walk a mile in their shoes.     Then when you do criticize them,   you will be a mile away and have their shoes !  
Joke 3:  Two Rednecks were seated at the end of a bar when a young lady seated a few stools up began to choke on a piece of hamburger.  She was turning blue and obviously in serious respiratory distress. One said to the other, "That gal there is having a bad time!"  The other agreed and said "Think we should go help?"  "You bet," said the first,and with that, he ran over and said, "Can you breathe??"  She shook her head no.  He said, "Can you speak??"  She again shook her head no. With that, he pulled up her skirt and licked her on the butt.  She was so shocked, she coughed up the obstruction and began to breathe-with great relief.  The redne

## **Evaluation**